In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim

from utils import seed
from data import dataset
from model.backbone import VGG16, ResNet34
from model.simclr import basic_simclr, data_aug, simclr_train
from model import basic_classifier
from utils import train

seed.seed_everything()

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


# VGG

## SimCLR

In [4]:
path = './data'

dataset = data_aug.ContrastiveLearningDataset(path=path, input_shape=96)

batch_size = 256

train_set = dataset.get_dataset()
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=0,
                                           pin_memory=True, 
                                           drop_last=True)

In [ ]:
backbone = VGG16.VGG16_Backbone(pretrain=False).to(device=device)
model = basic_simclr.BasicSimCLR(backbone).to(device=device)

print(model)

In [6]:
# default hyperparameters
lr = 3e-4
weight_decay = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                       T_max=len(train_loader), 
                                                       eta_min=0,
                                                       last_epoch=-1)

In [7]:
simclr = simclr_train.SimCLR(model=model,
                             model_name='VGG16',
                             scheduler=scheduler,
                             optimizer=optimizer,
                             device=device,
                             epochs=200)

In [ ]:
simclr.train(train_loader=train_loader)

In [ ]:
# log dir
print(simclr.writer.log_dir)

## Model Train

In [ ]:
# (already downloaded) dataset path 
path = './data'

train_loader, val_loader, test_loader = dataset.load_pcam(path=path, 
                                                          input_shape=96,
                                                          augmentation=True,
                                                          normalize=True,
                                                          batch_size=256,
                                                          seed=21)

In [15]:
backbone = VGG16.VGG16_Backbone(pretrain=False).to(device=device)
model_temp = basic_simclr.BasicSimCLR(backbone).to(device=device)

# path - VGG 
file_path = 'checkpoint_0002.pth.tar'

checkpoint_path = os.path.join(simclr.writer.log_dir, file_path)
checkpoint = torch.load(checkpoint_path, map_location=device)
model_temp.load_state_dict(checkpoint['state_dict'])

In [ ]:
# backbone from SimCLR
model = basic_classifier.BasicClassifier(backbone=backbone, 
                                         freezing=True, 
                                         num_classes=1).to(device=device)

print(model)

In [ ]:
EPOCHS = 50

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay=1e-3)
es = train.EarlyStopping(patience=EPOCHS//2, delta=0, mode='min', verbose=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=EPOCHS//5, gamma=0.9)

history = {'train_loss' : [],
           'val_loss': [],
           'train_accuracy': [],
           'val_accuracy': []}
           
max_loss = np.inf    

In [ ]:
for epoch in range(EPOCHS):
    train_loss, train_acc = train.model_train(model=model, 
                                              data_loader=train_loader, 
                                              criterion=criterion, 
                                              optimizer=optimizer, 
                                              device=device, 
                                              scheduler=scheduler, 
                                              tqdm_disable=False)
    val_loss, val_acc = train.model_evaluate(model=model, 
                                             data_loader=val_loader, 
                                             criterion=criterion, 
                                             device=device)
    
    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    es(val_loss)
    # Early Stop Check
    if es.early_stop:
        break

    if val_loss < max_loss:
        print(f'[INFO] val_loss has been improved from {max_loss:.5f} to {val_loss:.5f}. Save model.')
        max_loss = val_loss
        torch.save(model.state_dict(), 'Best_Model_VGG_3.pth')

    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, accuracy: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f} \n')

In [ ]:
train.plot_acc(history=history)

In [ ]:
train.plot_loss(history=history)

In [ ]:
backbone = VGG16.VGG16_Backbone(pretrain=False).to(device=device)
model = basic_classifier.BasicClassifier(backbone=backbone, 
                                         freezing=True, 
                                         num_classes=1).to(device=device)

model.load_state_dict(torch.load('Best_Model_VGG_3.pth', map_location=device))
model.eval()

test_loss, test_acc = train.model_evaluate(model=model,
                                           data_loader=test_loader,
                                           criterion=criterion,
                                           device=device)

print('Test Loss: %s'%test_loss)
print('Test Accuracy: %s'%test_acc)

# ResNet

## SimCLR

In [ ]:
path = './data'

dataset = data_aug.ContrastiveLearningDataset(path=path, input_shape=96)

batch_size = 256

train_set = dataset.get_dataset()
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers=0,
                                           pin_memory=True, 
                                           drop_last=True)

In [ ]:
backbone = ResNet34.ResNet34_Backbone(pretrain=False).to(device=device)
model = basic_simclr.BasicSimCLR(backbone).to(device=device)

print(model)

In [ ]:
# default hyperparameters
lr = 3e-4
weight_decay = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                       T_max=len(train_loader), 
                                                       eta_min=0,
                                                       last_epoch=-1)

In [ ]:
simclr = simclr_train.SimCLR(model=model,
                             model_name='ResNet34',
                             scheduler=scheduler,
                             optimizer=optimizer,
                             device=device,
                             epochs=200)

In [ ]:
simclr.train(train_loader=train_loader)

In [ ]:
# log dir
print(simclr.writer.log_dir)

## Model Train

In [ ]:
# (already downloaded) dataset path 
path = './data'

train_loader, val_loader, test_loader = dataset.load_pcam(path=path, 
                                                          input_shape=96,
                                                          augmentation=True,
                                                          normalize=True,
                                                          batch_size=256,
                                                          seed=21)

In [ ]:
backbone = ResNet34.ResNet34_Backbone(pretrain=False).to(device=device)
model_temp = basic_simclr.BasicSimCLR(backbone).to(device=device)

# path - ResNet 
file_path = 'checkpoint_0002.pth.tar'

checkpoint_path = os.path.join(simclr.writer.log_dir, file_path)
checkpoint = torch.load(checkpoint_path, map_location=device)
model_temp.load_state_dict(checkpoint['state_dict'])

In [ ]:
# backbone from SimCLR
model = basic_classifier.BasicClassifier(backbone=backbone, 
                                         freezing=True, 
                                         num_classes=1).to(device=device)

print(model)

In [ ]:
EPOCHS = 50

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, weight_decay=1e-3)
es = train.EarlyStopping(patience=EPOCHS//2, delta=0, mode='min', verbose=True)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=EPOCHS//5, gamma=0.9)

history = {'train_loss' : [],
           'val_loss': [],
           'train_accuracy': [],
           'val_accuracy': []}
           
max_loss = np.inf    

In [ ]:
for epoch in range(EPOCHS):
    train_loss, train_acc = train.model_train(model=model, 
                                              data_loader=train_loader, 
                                              criterion=criterion, 
                                              optimizer=optimizer, 
                                              device=device, 
                                              scheduler=scheduler, 
                                              tqdm_disable=False)
    val_loss, val_acc = train.model_evaluate(model=model, 
                                             data_loader=val_loader, 
                                             criterion=criterion, 
                                             device=device)
    
    history['train_loss'].append(train_loss)
    history['train_accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    es(val_loss)
    # Early Stop Check
    if es.early_stop:
        break

    if val_loss < max_loss:
        print(f'[INFO] val_loss has been improved from {max_loss:.5f} to {val_loss:.5f}. Save model.')
        max_loss = val_loss
        torch.save(model.state_dict(), 'Best_Model_ResNet_3.pth')

    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, accuracy: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f} \n')

In [ ]:
train.plot_acc(history=history)

In [ ]:
train.plot_loss(history=history)

In [ ]:
backbone = VGG16.VGG16_Backbone(pretrain=False).to(device=device)
model = basic_classifier.BasicClassifier(backbone=backbone, 
                                         freezing=True, 
                                         num_classes=1).to(device=device)

model.load_state_dict(torch.load('Best_Model_ResNet_3.pth', map_location=device))
model.eval()

test_loss, test_acc = train.model_evaluate(model=model,
                                           data_loader=test_loader,
                                           criterion=criterion,
                                           device=device)

print('Test Loss: %s'%test_loss)
print('Test Accuracy: %s'%test_acc)